#### Importando as bibliotecas

In [13]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb

#### Bases de dados

In [14]:
df_pedidos = pd.read_excel("data/pedidos.xlsx")
df_itens = pd.read_excel("data/itens.xlsx")
df_clientes = pd.read_excel("data/clientes.xlsx") 
df_produtos = pd.read_excel("data/produtos.xlsx") 
df_reviews = pd.read_excel("data/reviews.xlsx")
df_geolocalizacao = pd.read_excel("data/geolocalizacao.xlsx") 
df_pagamento = pd.read_excel("data/pagamento.xlsx") 
clientes_cluster = pd.read_excel("data/cluster_clientes.xlsx")

#### Fusão dos dataframes

In [15]:
clientes = clientes_cluster.merge(df_clientes, how="inner", on="cliente_id_unico").drop(columns=['Unnamed: 0', 'Unnamed: 0.1'])
clientes_pedidos = clientes.merge(df_pedidos, how="inner", on="cliente_id")
clientes_pedidos_itens = clientes_pedidos.merge(df_itens ,how="inner", on="pedido_id")
clientes_produtos = clientes_pedidos_itens.merge(df_produtos, how="inner", on="produto_id")

clientes_reviews = clientes_pedidos.merge(df_reviews, how="inner", on="pedido_id")
clientes_reviews["review_score"] = clientes_reviews["review_score"].astype(int)

df_all = clientes_produtos.merge(df_reviews, how="inner", on="pedido_id")
df_all = df_all.merge(df_pagamento, how="inner", on="pedido_id")

In [34]:
df_all['data_compra'] = pd.to_datetime(df_all['pedido_data_compra'], format='%d/%m/%y %H:%M:%S')
df_all['data_compra'] = df_all['data_compra'].dt.strftime('%d-%m-%Y')

In [35]:
df_all[['dia_compra','mes_compra', 'ano_compra']] = df_all['data_compra'].str.split('-', expand=True)

In [39]:
df_all.to_excel("data/df_all.xlsx")

In [16]:
clientes_reviews.to_excel("data/clientes_reviews.xlsx")
clientes_produtos.to_excel("data/clientes_produtos.xlsx")

---

### Analisando Cluster

#### Cluster 1

In [6]:
clientes_produtos[clientes_produtos.cluster == 1].describe()

,valor_total,qnt_pedidos,cluster,cliente_CEP,numero_de_itens,preco,preco_frete,produto_comprimento_nome,produto_comprimento_descricao,produto_quantidade_fotos,produto_peso_g,produto_largura_cm,produto_altura_cm,produto_comprimento_cm
count,7274.000000,7274.000000,7274.0,7274.000000,7274.000000,7274.000000,7274.000000,7177.000000,7177.000000,7177.000000,7271.000000,7271.000000,7271.000000,7271.000000
mean,331.005388,2.243745,1.0,33499.409128,1.269453,94.978627,18.505913,48.946774,725.245367,2.048210,1844.904965,30.148810,15.413561,23.159813
std,264.107489,0.945718,0.0,29781.154638,0.751539,99.254283,12.393608,9.989457,633.706110,1.596633,3444.674664,15.967035,12.282548,11.680527
min,35.940000,2.000000,1.0,1005.000000,1.000000,2.900000,0.000000,10.000000,4.000000,1.000000,50.000000,11.000000,2.000000,6.000000
25%,158.190000,2.000000,1.0,9470.000000,1.000000,36.850000,12.690000,42.000000,317.000000,1.000000,261.000000,18.000000,8.000000,15.000000
50%,249.900000,2.000000,1.0,22750.000000,1.000000,67.900000,15.855000,52.000000,544.000000,1.000000,650.000000,25.000000,12.000000,20.000000
75%,409.700000,2.000000,1.0,50940.000000,1.000000,119.000000,19.700000,57.000000,903.000000,3.000000,1600.000000,38.000000,20.000000,30.000000
max,2164.400000,16.000000,1.0,99750.000000,11.000000,1199.900000,196.940000,66.000000,3956.000000,15.000000,30000.000000,105.000000,105.000000,118.000000


**Cluster 1:** Grupo de clientes que **mais geraram faturamento** (*grupo estratégico*)

 - Máximo faturamento de R$ 13664
 
 
 - Mínimo faturamento de R$ 1146
 
 
 - Média de preço por cliente = R$ 1026,81
 
 
 - Quantidade máxima de pedidos = 4
 
 
 - No geral não são clientes que fizeram muitos pedidos
 
     - 75% dos clientes fizeram 1 pedido e escolheram um produto de R$ 1460
 
 
  

#### Analisando quais categorias geraram maior faturamento

In [7]:
clientes_produtos[clientes_produtos.cluster == 1].produto_categoria.value_counts().head(5)

cama_mesa_banho           1105
moveis_decoracao           779
esporte_lazer              672
beleza_saude               550
informatica_acessorios     526
Name: produto_categoria, dtype: int64

**Plano para uma campanha de marketing:**

Dentro da lógica do sistema, cabe ser feito uma recomendação, para esse tipo de cliente, de produtos dessas categorias que geraram o maior faturamento. Isso pode ser feito via email, via sms, via rede social, estimulando o cliente a fazer mais pedidos dentro dessas categorias mais comuns da Via Sudeste.   


#### Analisando as reviews

**Legenda** 

1 => Uma estrela (Muito insatisfeito)

2 => Duas estrelas (Insatisfeito)

3 => Três estrelas

4 => Duas estrelas (Satisfeito) 

5 => Cuinco estrelas (Muito Satisfeito)

In [8]:
clientes_reviews[clientes_reviews.cluster == 1].describe()

,valor_total,qnt_pedidos,cluster,cliente_CEP,review_score
count,6435.000000,6435.00000,6435.0,6435.000000,6435.000000
mean,302.216872,2.23512,1.0,33946.508625,4.148407
std,238.771568,0.94677,0.0,29728.716981,1.319097
min,35.940000,2.00000,1.0,1005.000000,1.000000
25%,147.570000,2.00000,1.0,9760.500000,4.000000
50%,227.500000,2.00000,1.0,22793.000000,5.000000
75%,369.720000,2.00000,1.0,52765.000000,5.000000
max,2164.400000,16.00000,1.0,99750.000000,5.000000


- **Metade dos clientes** (50%) **está muito satisfeita** com seus pedidos. 


- Aqui cabe uma **recomendação de produtos** dentro das categorias vistas acima, já que ficaram muito satisfeitos com seus pedidos. Isso dentro desse grupo estratégico dos clientes que geraram maior faturamento. 


- Além disso, **captar os comentários** de reviews de ambos os extremos de satisfação. Reviews negativos mostram o que dá para ser melhorado. Reviews positivos mostram o que ta dando certo. 


- Para os clientes que não tiveram experiência positiva, caso seja do esforço da empresa, contatá-los para avaliar e **resolver os problemas criados**. Dessa forma, recuperamos aqueles clientes que não voltariam a comprar na Via Sudeste. 

---

#### Cluster 2

In [9]:
clientes_reviews[clientes_reviews.cluster == 2].describe()

,valor_total,qnt_pedidos,cluster,cliente_CEP,review_score
count,8170.000000,8170.0,8170.0,8170.000000,8170.000000
mean,510.826830,1.0,2.0,39350.373439,3.945043
std,198.392692,0.0,0.0,29859.510648,1.463655
min,311.120000,1.0,2.0,1006.000000,1.000000
25%,358.032500,1.0,2.0,13220.000000,3.000000
50%,433.135000,1.0,2.0,30285.000000,5.000000
75%,617.172500,1.0,2.0,65698.750000,5.000000
max,1150.800000,1.0,2.0,99930.000000,5.000000


Grupo de clientes que **mais fizeram pedidos** 

 - 25% dos pedidos que os clientes fizeram tiveram score 4.

 - Isso justifica o fato deles serem os clientes que fizeram mais pedidos, pois tiveram a maior satisfação. 

 - A chance deles voltarem a comprar é bem alta. 
 
  

---

#### Cluster 0

In [10]:
clientes_produtos[clientes_produtos.cluster == 0].produto_categoria.value_counts()

cama_mesa_banho                  9225
beleza_saude                     7785
esporte_lazer                    7211
moveis_decoracao                 6640
informatica_acessorios           6257
                                 ... 
la_cuisine                          9
fashion_roupa_infanto_juvenil       7
pc_gamer                            5
seguros_e_servicos                  2
pcs                                 1
Name: produto_categoria, Length: 73, dtype: int64

Apesar de serem clientes que **não geraram grande faturamento**, fizeram uma **grande quantidade de compras** e categorias. 

Aqui cabe uma **recomendação de produtos** nessas categorias para esse tipo de cliente, de forma que **estimule** esses clientes a **fazerem mais de um pedido**, ja que estão com um nível de satisfação interessante.

#### Cluster 3

In [11]:
clientes_produtos[clientes_produtos.cluster == 3].produto_categoria.value_counts()

informatica_acessorios                            144
pcs                                               130
beleza_saude                                      105
automotivo                                         99
relogios_presentes                                 98
moveis_escritorio                                  91
moveis_decoracao                                   73
esporte_lazer                                      54
ferramentas_jardim                                 46
construcao_ferramentas_construcao                  44
utilidades_domesticas                              44
instrumentos_musicais                              42
eletroportateis                                    38
bebes                                              38
telefonia                                          34
cama_mesa_banho                                    29
eletrodomesticos_2                                 26
telefonia_fixa                                     24
agro_industria_e_comercio   

In [12]:
clientes_reviews[clientes_reviews.cluster == 3].describe()

,valor_total,qnt_pedidos,cluster,cliente_CEP,review_score
count,847.000000,847.000000,847.0,847.000000,847.000000
mean,1813.110720,1.085006,3.0,42061.480519,3.912633
std,870.922006,0.336649,0.0,30017.359793,1.535065
min,1153.090000,1.000000,3.0,1020.000000,1.000000
25%,1333.400000,1.000000,3.0,14805.500000,3.000000
50%,1544.820000,1.000000,3.0,35447.000000,5.000000
75%,2026.540000,1.000000,3.0,69250.500000,5.000000
max,13664.080000,4.000000,3.0,99615.000000,5.000000


Recomendação igual ao cluster de clientes com maior faturamento